In [4]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler
from gensim import corpora, models

In [5]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler
from gensim import corpora, models

# Load breast cancer dataset
data = load_breast_cancer()
features = data.data
labels = data.target

# Min-Max scaling on features
scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(features)

# Number of clients
num_clients = 2

# Prepare data for Gensim LDA
corpus = [list(enumerate(doc)) for doc in normalized_features]

# Split features into two groups using LDA
dictionary = corpora.Dictionary([range(features.shape[1])])
lda_model = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=features.shape[1],
    alpha=0,  # Set alpha to 0 for non-IID distribution
    random_state=42,
)

# Generate non-IID data for each client
non_iid_data = []
for client_id in range(num_clients):
    topic_distribution = np.random.dirichlet(
        np.ones(features.shape[1]) * 10, size=1
    ).flatten()
    document = lda_model.make_doc(
        [
            (i, topic_distribution[i] * features.shape[0])
            for i in range(features.shape[1])
        ]
    )
    non_iid_data.append(document)

# Display the non-IID data
for client_id, document in enumerate(non_iid_data):
    print(f"Client {client_id + 1} data:")
    print(
        lda_model.print_topic(
            lda_model.get_document_topics(
                document, minimum_probability=0.0, per_word_topics=False
            ),
            num_words=features.shape[1],
        )
    )

# Now 'non_iid_data' contains non-IID data for each client based on LDA

TypeError: decoding to str: need a bytes-like object, int found